In [6]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import requests
from StringIO import StringIO
import urllib

import sys
import os

caffe_root = '/home/jonnatas/workspace_python/caffe/'
sys.path.insert(0, caffe_root + 'python')

print caffe_root

import caffe


/home/jonnatas/workspace_python/caffe/


In [7]:
caffe.set_mode_cpu()

model_def = os.path.join(caffe_root, 'models', 'bvlc_reference_caffenet','deploy.prototxt')
model_weights = os.path.join(caffe_root, 'models','bvlc_reference_caffenet','bvlc_reference_caffenet.caffemodel')

net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)

mu = np.load(os.path.join(caffe_root, 'python','caffe','imagenet','ilsvrc_2012_mean.npy'))
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

net.blobs['data'].reshape(50,        # batch size
                          3,         # 3-channel (BGR) images
                          227, 227)  # image size is 227x227



mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]


In [8]:
def printFaces(faces,coordinates):
    print('Total of faces detected = '+str(len(faces)))
    
    print 'coordinates: '
    for coordinate in coordinates:
        x,y,w,h = coordinate
        print [(x,y), (x+w,y+h), (x+w, y), (x, y+h)]
        print '\n'

In [9]:
def contFaces(faces,frame,DOWNSCALE):
    coordinates = []
    # print 'face detected!'
    for i in faces:
        x, y, w, h = [ v*DOWNSCALE for v in i ]

        coordinates.append((x,y,w,h))
        print x,y,w,h
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0))
    
    return coordinates
    

In [10]:
def detect(frame):
    height, width, depth = frame.shape
   
    # create grayscale version
    grayscale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # equalize histogram
    cv2.equalizeHist(grayscale, grayscale)
    
    # detect objects
    classifier = cv2.CascadeClassifier("/usr/local/share/OpenCV/haarcascades/haarcascade_frontalface_alt.xml")
    
    DOWNSCALE = 4
    minisize = (frame.shape[1]/DOWNSCALE,frame.shape[0]/DOWNSCALE)
    miniframe = cv2.resize(frame, minisize)
    faces = classifier.detectMultiScale(miniframe)
    coordinates = []
    
    if len(faces)>0:
        coordinates = contFaces(faces,frame,DOWNSCALE)
        # print 'face detected!'
        
    printFaces(faces, coordinates)
    
    return frame

In [11]:
def get_image_url(url):
        cap = cv2.VideoCapture(url)
        ret,img = cap.read()

        return img

In [12]:

def get_image_camera():

        cap = cv2.VideoCapture(0)
        if (cap==True):
            print("deu certo")
            
        while(True):
            # Capture frame-by-frame
            ret, frame = cap.read()
            img = frame.copy()
            # Call the function
            frame = detect(frame)

            # Display the resulting frame
            cv2.imshow('frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cv2.imwrite("frame.jpg", frame)
        cv2.imwrite(os.path.join(caffe_root, 'examples', 'images','frame.jpg'), frame)

        
        # When everything done, release the capture
        cap.release()
        cv2.destroyAllWindows()
        cap = cv2

        return frame

In [13]:
def cats_and_dogs_probability(output_prob, labels, path_animal):
        labels_animal = np.loadtxt(path_animal, str, delimiter='\t')
        
        index = 0
        list_animal = []
        probability_animal = 0.0
        
        for i in labels:
                if i in labels_animal:
                        list_animal.append((output_prob[index],labels[index]))
                        probability_animal += output_prob[index]
                index += 1
        list_animal = sorted(list_animal, reverse=True)
        
        for i in list_animal:
            probability , animal_class = i
            print 'Synset ' + str(animal_class)
            print 'Probability ' + str(probability * 100.0) + "%"
        
        return probability_animal

In [14]:
def probability(image):    
        transformed_image = transformer.preprocess('data', image)
        plt.imshow(image)
        plt.axis('off')

        # copy the image data into the memory allocated for the net
        net.blobs['data'].data[...] = transformed_image

        ### perform classification
        output = net.forward()

        output_prob = output['prob'][0]  # the output probability vector for the first image in the batch

        # print 'predicted class is:', output_prob.argmax()
        
        # load ImageNet labels
        labels_file = caffe_root + 'data/ilsvrc12/synset_words.txt'
   
        labels = np.loadtxt(labels_file, str, delimiter='\t')

        dogs_file = os.path.join(caffe_root, 'data','ilsvrc12','dogs.txt')
        cats_file = os.path.join(caffe_root, 'data', 'ilsvrc12','cats.txt')

        top_inds = output_prob.argsort()[::-1][:]

        probability_dogs = cats_and_dogs_probability(output_prob, labels, dogs_file)
        probability_cats = cats_and_dogs_probability(output_prob, labels, cats_file)
               
        print '\n\n::Results::\n'
        print 'Feline probability: ' + str( (probability_cats) * 100.0 ) + "%"
        print 'Canine probability: ' + str( (probability_dogs) * 100.0 ) + "%"
        print 'Most probable synset:', labels[output_prob.argmax()]
        print 'Compatibility: ' + str(output_prob[output_prob.argmax()] * 100.0) + "%"
        print '\n'


In [15]:
def options():
        print('1 - image')
        print('2 - url')
        print('3 - camera')
        print('4 - exit')

        return int(raw_input(''))

In [16]:
def option_image():
        image_name = raw_input('File name:\n')

        image = cv2.VideoCapture(image_name)
        ret, img = image.read()

        # detect(img)

        image_recognition = caffe.io.load_image(image_name)

        probability(image_recognition)
        image_detected = detect(img)

        plt.imshow(image_detected[:,:,::-1])
        plt.title('Detection result')
        plt.axis('off')
        plt.show()

In [17]:
def option_url():
        image_url = raw_input('enter the image url:\n')
        image = get_image_url(image_url)

        urllib.urlretrieve(image_url, os.path.join(caffe_root, 'examples', 'images','url.jpg'))
        image_recognition = caffe.io.load_image(os.path.join(caffe_root, 'examples', 'images','url.jpg'))
        # print image

        probability(image_recognition)
        image_detected = detect(image)

        plt.imshow(image_detected[:,:,::-1])
        plt.title('Detection result')
        plt.axis('off')
        plt.show()

In [18]:
def option_camera():
        frame = get_image_camera()
        image_recognition = caffe.io.load_image(os.path.join(caffe_root, 'examples', 'images','frame.jpg'))
        probability(image_recognition)
        image_detected = detect(frame)

        plt.imshow(image_detected[:,:,::-1])
        plt.title('Detection result')
        plt.axis('off')
        plt.show()

In [19]:

if __name__ == '__main__':

        while True:
                option = options()

                if option == 1:
                    option_image()

                elif option == 2:
                     option_url()

                elif option == 3:
                    option_camera()

                elif option == 4:
                    break

1 - image
2 - url
3 - camera
4 - exit
3
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordinates: 
Total of faces detected = 0
coordina

#### 